In [35]:
import sys
print(sys.version)

3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]


In [2]:
pip install pulp


     --------------------------------------- 17.7/17.7 MB 12.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [29]:
#Example Implementation in Python  EX1
#Using a simple optimization framework, we can illustrate this model using the PuLP library:
import pulp

# Parameters
N_b = 4  # Number of beams
S = 100  # Size of substrate (mm²)
v = 10   # Writing speed (mm/s)
t_e = 0.1  # Exposure time (s per pixel)
d = 0.01  # Defect rate (probability)
M = 1000  # Total number of pixels in design

# Create a linear programming problem
problem = pulp.LpProblem("Multi_Beam_Mask_Writer", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts('write', (range(N_b), range(M)), cat='Binary')

# Objective function
problem += S / (N_b * v) + pulp.lpSum(t_e * x[i][j] for i in range(N_b) for j in range(M)) + pulp.lpSum(d * x[i][j] for i in range(N_b) for j in range(M))

# Constraints
for j in range(M):
    problem += pulp.lpSum(x[i][j] for i in range(N_b)) <= 1  # Each pixel can only be written by one beam

# Data transfer rate constraint
problem += S / v <= 50  # Example threshold for data transfer rate

# Solve the problem
problem.solve()


# Output the results
print("Status:", pulp.LpStatus[problem.status])  # Output the status of the optimization


# Check if the problem was solved successfully
if pulp.LpStatus[problem.status] == "Optimal":
    for i in range(N_b):
        for j in range(M):
            if pulp.value(x[i][j]) == 1:
                print(f"Beam {i} writes to pixel {j}")
else:
    print("No feasible solution found.")

Status: Optimal


In [30]:
#Example Implementation in Python  EX2
import pulp

# Parameters
N_b = 4  # Number of beams
S = 100  # Size of substrate (mm²)
v = 10   # Writing speed (mm/s)
t_e = 0.1  # Exposure time (s per pixel)
d = 0.01  # Defect rate (probability)
M = 1000  # Total number of pixels in design

# Create a linear programming problem
problem = pulp.LpProblem("Multi_Beam_Mask_Writer", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts('write', (range(N_b), range(M)), cat='Binary')

# Objective function
problem += S / (N_b * v) + pulp.lpSum(t_e * x[i][j] for i in range(N_b) for j in range(M)) + pulp.lpSum(d * x[i][j] for i in range(N_b) for j in range(M))

# Constraints
for j in range(M):
    problem += pulp.lpSum(x[i][j] for i in range(N_b)) <= 1  # Each pixel can only be written by one beam

# Data transfer rate constraint
problem += S / v <= 50  # Example threshold for data transfer rate

# Solve the problem
problem.solve()

# Output the results
for i in range(N_b):
    for j in range(M):
        if pulp.value(x[i][j]) == 1:
            print(f"Beam {i} writes to pixel {j}")

# Print the status of the problem
print("Status:", pulp.LpStatus[problem.status])



Status: Optimal


In [32]:
#Example Implementation in Python  EX3

#If you are still getting the same output with "Status: Optimal" and no pixels being written to,it indicates that 
#the constraints are too strict or the model setup is not allowing for any beams to be assigned to the pixels.
#Here are a few more things you can try to find a feasible solution:

#Remove Constraints Temporarily: Comment out the constraints one by one to identify which one is causing the issue. 
#Start by commenting out the pixel assignment constraint and the data transfer constraint.

#Check Objective Function: Simplify the objective function to ensure it’s set up correctly. 
#If your goal is to minimize the time to write, you might want to focus only on exposure time or only on defect rates.

#Increase M: Experiment with increasing the number of pixels (M) to see if that changes the output.

#Modify Parameters: Adjust other parameters such as the writing speed (v) or exposure time (t_e) to see if they allow for
#a feasible solution.

import pulp

# Parameters
N_b = 4  # Number of beams
S = 100  # Size of substrate (mm²)
v = 10   # Writing speed (mm/s)
t_e = 0.1  # Exposure time (s per pixel)
d = 0.01  # Defect rate (probability)
M = 3  # Total number of pixels in design, reduced for testing

# Create a linear programming problem
problem = pulp.LpProblem("Multi_Beam_Mask_Writer", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts('write', (range(N_b), range(M)), cat='Binary')

# Objective function: Simplified for testing
problem += pulp.lpSum(t_e * x[i][j] for i in range(N_b) for j in range(M)) 

# Constraints
for j in range(M):
    problem += pulp.lpSum(x[i][j] for i in range(N_b)) <= 1  # Each pixel can only be written by one beam

# Temporarily remove other constraints to find a feasible solution
# problem += S / v <= 50  # Example threshold for data transfer rate

# Solve the problem
problem.solve()

# Output the results
print("Status:", pulp.LpStatus[problem.status])  # Output the status of the optimization

# Check if the problem was solved successfully
if pulp.LpStatus[problem.status] == "Optimal":
    for i in range(N_b):
        for j in range(M):
            if pulp.value(x[i][j]) == 1:
                print(f"Beam {i} writes to pixel {j}")
else:
    print("No feasible solution found.")

# Output the decision variable values
print("\nDecision Variables:")
for i in range(N_b):
    for j in range(M):
        print(f"x[{i}][{j}] = {pulp.value(x[i][j])}")


Status: Optimal

Decision Variables:
x[0][0] = 0.0
x[0][1] = 0.0
x[0][2] = 0.0
x[1][0] = 0.0
x[1][1] = 0.0
x[1][2] = 0.0
x[2][0] = 0.0
x[2][1] = 0.0
x[2][2] = 0.0
x[3][0] = 0.0
x[3][1] = 0.0
x[3][2] = 0.0


how can i understand problem.solve() doesnt have andy feasible solution
When you run problem.solve() in PuLP, the method returns an optimal solution if one exists, but if the solution is not feasible, it won't necessarily indicate that directly. Here are ways to understand whether the problem has a feasible solution:

Checking Feasibility
Status Check: After calling problem.solve(), check the status of the problem:

python code
print("Status:", pulp.LpStatus[problem.status])
If the status is Infeasible, then the problem has no feasible solution under the current constraints.

Constraints Review: If the status is not Optimal, you can investigate the constraints you've set up:

Make sure they aren’t overly restrictive.
Comment out constraints one by one to see if the model becomes feasible when certain constraints are removed.
Solver Output: Some solvers provide detailed output regarding why a problem is infeasible. You can enable the solver's output by setting pulp.LpSolverDefault.msg to True:

python
Copy code
pulp.LpSolverDefault.msg = True  # Enable solver output
problem.solve()
Variable Values: After solving, check the values of the decision variables:

python
Copy code
for i in range(N_b):
    for j in range(M):
        print(f"x[{i}][{j}] = {pulp.value(x[i][j])}")
If all values are zero (or the expected values based on your constraints), it indicates no feasible solution was found.

Feasibility Relaxation: You can also relax some constraints to see if that leads to a feasible solution. For example, if you have constraints that require certain conditions, see if removing or loosening them leads to a solution.
